In [216]:
from collections import Counter
import random
import bisect
import numpy as np

In [217]:
text = open("berp-training.txt").read()
words = text.split()
freqs = Counter(words)
N = sum(freqs.values())

Generating the Unigram

In [218]:
#replace all 1s with unk
def addUNK(data):
    count = 0
    for key,value in data.items():
        if value == 1:
            remove = dict(data)
            count +=1
    del remove[key]
    data["<unk>"] = count
    return data
uni = addUNK(freqs)
uni

Counter({'<s>': 6756,
         "let's": 236,
         'start': 438,
         'over': 414,
         '</s>': 6756,
         'my': 36,
         'mother': 2,
         'is': 359,
         'coming': 3,
         'to': 2153,
         'visit': 3,
         'and': 261,
         "i'd": 508,
         'like': 1221,
         'take': 28,
         'her': 1,
         'dinner': 362,
         'new': 4,
         'query': 3,
         'now': 49,
         "i'm": 246,
         'interested': 42,
         'in': 312,
         'some': 323,
         'middle': 17,
         'eastern': 15,
         'food': 992,
         'oh': 51,
         'i': 2264,
         'have': 594,
         'breakfast': 127,
         'increase': 2,
         'the': 937,
         'walking': 56,
         'distance': 165,
         'can': 381,
         'go': 390,
         'fifteen': 110,
         'minutes': 209,
         'from': 205,
         'icsi': 156,
         'would': 652,
         'french': 87,
         'okay': 152,
         'back': 23,
       

In [219]:
def product(nums):
    prod = 1
    for num in nums: prod = prod * num
    return prod

def unigramModel(word): 
    return freqs[word]/N

def sentenceProb(words):
    return product(unigramModel(word) for word in words)

In [220]:
#write the probs from test sentences to a txt file
def uniFile(filename):
    f = open(filename)
    lines = f.readlines()
    r = open("ostlund-alison-assgn2-unigram-out.txt", 'w')
    for line in lines:
        test = line.split()
        prob = sentenceProb(test)
        r.write(str(prob) + "\n")
    f.close()
    r.close()
    
uniFile("berp-100-test.txt")

Generating the Bigram

In [221]:
#creating bigram,replace words from unigram with count of 1 with unk
bigram = {}
for i in range(len(words)-1):
    w1 = words[i]
    w2 = words[i+1]
    if w1 not in (uni):
        w1 = "<unk>"
    if w2 not in (uni):
        w2 = "<unk>"  
    if (w1, w2) not in bigram:
        bigram[(w1, w2)] = 1
    else:
        bigram[(w1, w2)] += 1
    
bigram

{('<s>', "let's"): 196,
 ("let's", 'start'): 170,
 ('start', 'over'): 403,
 ('over', '</s>'): 367,
 ('</s>', '<s>'): 6755,
 ('<s>', 'my'): 6,
 ('my', 'mother'): 2,
 ('mother', 'is'): 1,
 ('is', 'coming'): 1,
 ('coming', 'to'): 1,
 ('to', 'visit'): 2,
 ('visit', 'and'): 1,
 ('and', "i'd"): 5,
 ("i'd", 'like'): 495,
 ('like', 'to'): 953,
 ('to', 'take'): 12,
 ('take', 'her'): 1,
 ('her', 'to'): 1,
 ('to', 'dinner'): 12,
 ('dinner', '</s>'): 224,
 ('<s>', 'new'): 1,
 ('new', 'query'): 1,
 ('query', '</s>'): 2,
 ('<s>', 'now'): 7,
 ('now', "i'm"): 8,
 ("i'm", 'interested'): 35,
 ('interested', 'in'): 39,
 ('in', 'some'): 5,
 ('some', 'middle'): 5,
 ('middle', 'eastern'): 14,
 ('eastern', 'food'): 10,
 ('food', '</s>'): 670,
 ('<s>', 'oh'): 29,
 ('oh', 'i'): 11,
 ('i', 'have'): 91,
 ('have', 'to'): 17,
 ('to', 'breakfast'): 1,
 ('breakfast', '</s>'): 89,
 ('i', 'increase'): 1,
 ('increase', 'the'): 2,
 ('the', 'walking'): 3,
 ('walking', 'distance'): 47,
 ('distance', 'i'): 3,
 ('i', 'can')

In [222]:
print(sum(bigram.values()))

56695


In [223]:
#Creating bigram probabilites 
#Taking the frequencies of each bigram, add one to each for add-1 smoothing 
# Dividing by prefix frequency in the unigram + unigram count for add-1 smoothing
def bigramModel(bgram):
    biprob = 1
#     bgram = sentence.split()
    for i in range(len(bgram) -1):
        if (bgram[i], bgram[i+1]) not in bigram:
            top = 1
        else:
            top = bigram[bgram[i], bgram[i+1]] + 1
        if (bgram[i] not in uni):
            bottom = 1
        else:
            bottom = uni[bgram[i]] + len(uni)
        biprob = biprob * (top/bottom)
    return biprob  

In [224]:
#write the probs from test sentences to a txt file
def biFile(filename):
    f = open(filename)
    lines = f.readlines()
    r = open("ostlund-alison-assgn2-bigram-out.txt", 'w')
    for line in lines:
        test = line.split()
        prob = bigramModel(test)
        r.write(str(prob) + "\n")
    f.close()
    r.close()
    
biFile("berp-100-test.txt")

Random Sentence Generation

In [214]:
#random sampling taken from github Random Sample Unigrams.ipynb
def getSampleBins(wordprobs):
    index = 0; bins = []
    for w in wordprobs:
        index += w
        bins.append(index)
    return bins

def sampleWords(bins):
    rndIndex = random.random()
    return bisect.bisect(bins, rndIndex)

In [245]:
#shannons method
def shannonsMethod():
    file = open("ostlund-alison-bigram-rand-corpus.txt", "w")
    arr = {}
    for key, value in bigram.items():
        first = key[0]
        if first in arr:
            arr[first].append((key[1], value))
        else:
            arr[first] = [(key[1], value)]
#generate 100 sentences
    for i in range(100):
        shannon = []
        shannon.append("<s>")
        start = list(arr["<s>"][i][1]/ freqs["<s>"] for i in range(len(arr["<s>"])))
        shannon.append(arr["<s>"][sampleWords(getSampleBins(start))][0])
        #fill in rest of sentence
        word = 1
        while(shannon[-1] != "</s>"):
            curr = shannon[word]
            add = list(arr[curr][i][1]/freqs[curr] for i in range(len(arr[curr])))
            shannon.append(arr[curr][sampleWords(getSampleBins(add))][0])
            word += 1
        shannon = " ".join(shannon)
        print(shannon)
        file.write(shannon)
        file.write("\n")
    file.close()

shannonsMethod()      
    


<s> i'm willing to go on saturday </s>
<s> might be less than fifteen miles please show me the restaurant that lilly's </s>
<s> can you show me about hong-kong villa </s>
<s> lalime's on friday lunchtime cheap food </s>
<s> what about cactus taqueria </s>
<s> let's start over </s>
<s> can you gave me more information about um dressy casual </s>
<s> she likes italian food oh i'd like to get cheap cafe bistro </s>
<s> what about the most twenty minutes from icksee should be within twenty dollars </s>
<s> about indian food restaurants on friday or french food is good chopped liver </s>
<s> distance from five minutes </s>
<s> the bay area </s>
<s> maxim </s>
<s> let's start over </s>
<s> i'd like </s>
<s> i'm interested in berkeley </s>
<s> is no less than twenty dollars </s>
<s> i'd like to eat in berkeley </s>
<s> where can i don't want american food are there any price </s>
<s> it doesn't matter i don't want to ten minutes from icsi </s>
<s> american food </s>
<s> i'm looking for dinner